# Scripts for Scaling IR Data

### Import Necessary Packages

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time as datetime_time
import os
import pandas as pd
from datetime import datetime, timedelta
import time

### Small Functions Used in Main Function

In [10]:
def convert_string_time_to_seconds(time_value):
    if isinstance(time_value, str):  # If time is a string
        try:
            # Handle "HH:MM:SS" or "HH:MM:SS.S" format (with or without fractional seconds)
            if len(time_value.split(":")) == 3:
                try:
                    # First, try to parse with fractional seconds
                    time_obj = datetime.strptime(time_value, '%H:%M:%S.%f')
                except ValueError:
                    # If that fails, try parsing without fractional seconds
                    time_obj = datetime.strptime(time_value, '%H:%M:%S')
                return timedelta(hours=time_obj.hour, minutes=time_obj.minute, seconds=time_obj.second, microseconds=time_obj.microsecond).total_seconds()

            # Handle "MM:SS" or "MM:SS.S" format (with or without fractional seconds)
            elif len(time_value.split(":")) == 2:
                try:
                    # First, try to parse with fractional seconds
                    time_obj = datetime.strptime(time_value, '%M:%S.%f')
                except ValueError:
                    # If that fails, try parsing without fractional seconds
                    time_obj = datetime.strptime(time_value, '%M:%S')
                return timedelta(minutes=time_obj.minute, seconds=time_obj.second, microseconds=time_obj.microsecond).total_seconds()
        
        except ValueError:
            return np.nan  # Handle invalid formats
    
    elif isinstance(time_value, datetime.time):  # If time is already a datetime.time object
        return timedelta(hours=time_value.hour, minutes=time_value.minute, seconds=time_value.second, microseconds=time_value.microsecond).total_seconds()
    
    return np.nan  # Return NaN for anything else

def datetime_to_seconds(value):
    """
    Convert a single time or datetime value to total seconds.
    Handles datetime.datetime, datetime.time, and strings.
    """
    if isinstance(value, datetime):
        # If the value is a datetime object, calculate total seconds from the first time in the series
        base_date = datetime(1900, 1, 1)  # Start counting from the "start" date
        total_seconds = (value - base_date).total_seconds()
    elif isinstance(value, str):
        # Parse the string into a datetime object, with two possible formats
        try:
            # Try time-only format: "HH:MM:SS.sss"
            dt_value = datetime.strptime(value, "%H:%M:%S.%f")
        except ValueError:
            # If the time-only parsing fails, try full datetime format: "YYYY-MM-DD HH:MM:SS.sss"
            dt_value = datetime.strptime(value, "%Y-%m-%d %H:%M:%S.%f")
        
        # In either case, we treat the full datetime as starting from base_date (1900-01-01)
        base_date = datetime(1900, 1, 1)
        total_seconds = (dt_value - base_date).total_seconds()
    elif isinstance(value, time):
        # If the value is a time object, treat it as relative to the start of the first day
        base_time = datetime.strptime("00:00:00.000000", "%H:%M:%S.%f")  # Reference to start of the first day
        total_seconds = (datetime.combine(datetime(1900, 1, 1), value) - base_time).total_seconds()
    elif isinstance(value, timedelta):
        # If it's a timedelta object, just extract total seconds directly
        total_seconds = value.total_seconds()
    else:
        raise ValueError(f"Unsupported type: {type(value)}")
    
    return total_seconds

def convert_time_to_seconds(time_series):
    # Apply the conversion function to each element in the series
    times_2=time_series.apply(datetime_to_seconds)
    times_corrected=[]
    day_counter=0
    for i,j in enumerate(times_2):
        if i!=0 and times_2[i]<times_2[i-1]:
            day_counter+=1
        times_corrected.append(j+24*60*60*day_counter)
    return pd.Series(times_corrected)

# Function to dynamically detect which column is starting material and which is product based on trends
def detect_starting_material_and_product(ir_starting_material, ir_product):
    # Check initial and final trends for both columns
    if ir_starting_material.iloc[0] > ir_starting_material.iloc[-1]:
        # If starting material is decreasing, it is the starting material
        starting_material_ir = ir_starting_material
        product_ir = ir_product
    else:
        # Otherwise, product is increasing and starting material is increasing
        starting_material_ir = ir_product
        product_ir = ir_starting_material

    return starting_material_ir, product_ir

# Function to process the IR data for starting material and product
def process_ir_data(time_in_seconds, ir_starting_material, ir_product, reaction_start_time, ir_start_time, initial_conc_starting_material, initial_conc_second_material,Name_SM1,Name_SM2,ArCl=False):
    # Convert IR start time to seconds
    ir_start_seconds = convert_string_time_to_seconds(ir_start_time)
    # Convert reaction start time to seconds
    reaction_start_seconds = convert_string_time_to_seconds(reaction_start_time)
    
    # Calculate time difference between reaction start and IR experiment start (in seconds)
    time_difference = reaction_start_seconds - ir_start_seconds

    print("Time difference between IR start and reaction start (in seconds):", time_difference)
    
    # Convert the time column to seconds and adjust it by subtracting the time difference
    
    adjusted_time_in_seconds = time_in_seconds + time_difference
    
    # Convert adjusted time to minutes
    time_in_minutes = adjusted_time_in_seconds / 60

    # Calculate the time after addition
    time_after_addition_minutes = time_in_minutes
    print("Time after addition in minutes:", time_after_addition_minutes)
    
    # Convert IR data columns to numeric, handle invalid data with NaN
    ir_starting_material = pd.to_numeric(ir_starting_material, errors='coerce')
    ir_product = pd.to_numeric(ir_product, errors='coerce')

    # Dynamically detect which column is the starting material and which is the product
    starting_material_ir, product_ir = detect_starting_material_and_product(ir_starting_material, ir_product)
    
    # Process starting material IR trend
    avg_final_starting_material_ir = np.mean(starting_material_ir[-10:])  # average final 10 values
    
    # Get the last 3 values of starting material IR before the reaction started (time_after_addition_minutes < 0)
    print('time',time_after_addition_minutes)
    print( starting_material_ir[time_after_addition_minutes < 0])
    
    valid_starting_material_before_reaction = starting_material_ir[time_after_addition_minutes < 0].tail(3).values
    print('Starting Material IR before reaction:', valid_starting_material_before_reaction)
    if len(valid_starting_material_before_reaction) < 3:
        raise ValueError("There are less than 3 valid values before the reaction started. Please check your data.")
    
    # Ensure no NaN values before averaging
    if np.isnan(valid_starting_material_before_reaction).any():
        raise ValueError("Starting material IR contains NaN values before the reaction starts. Please check your data.")
    
    # Average the last 3 values before reaction (when adjusted time is negative)
    avg_initial_starting_material_ir = np.mean(valid_starting_material_before_reaction)

    if pd.isna(avg_initial_starting_material_ir) or avg_initial_starting_material_ir == 0:
        raise ValueError(f"Invalid initial starting material IR average: NaN or zero detected.\n"
                         f"Avg Initial IR: {avg_initial_starting_material_ir}")

    avg_initial_starting_material_ir = float(avg_initial_starting_material_ir)
    initial_conc_starting_material = float(initial_conc_starting_material)
    
    # Perform the scaling operation for starting material IR
    scaled_starting_material_ir = (starting_material_ir - avg_final_starting_material_ir) * (initial_conc_starting_material / (avg_initial_starting_material_ir - avg_final_starting_material_ir))
    print('Scaled Starting Material IR:', scaled_starting_material_ir)
    
    # Process product IR trend
    valid_product_before_reaction = product_ir[time_after_addition_minutes < 0].tail(3)
    if valid_product_before_reaction.isnull().any():
        raise ValueError("Product IR contains NaN values before the reaction starts. Please check your data.")

    avg_initial_product_ir = np.mean(valid_product_before_reaction)
    print('avg_initial_product_ir:', avg_initial_product_ir)
    
    avg_final_product_ir = np.mean(product_ir[-10:])
    print('avg_final_product_ir:', avg_final_product_ir)
    
    if pd.isna(avg_final_product_ir) or avg_final_product_ir == 0:
        raise ValueError("Invalid final product IR average: NaN or zero detected.")

    scaled_product_ir = (product_ir - avg_initial_product_ir) * (initial_conc_starting_material / (avg_final_product_ir - avg_initial_product_ir))

    # Create DataFrame for Sheet 1 (raw IR data with time adjustments)
    sheet1_data = pd.DataFrame({
        'Time (HH:MM:SS)': time_in_seconds/60,  # Keeping the original time string or time object
        'Time (minutes)': time_in_minutes,
        'Time after addition (minutes)': time_after_addition_minutes,
        'Starting Material IR': starting_material_ir,
        'Product IR': product_ir
    })

    # Create DataFrame for Sheet 2 (processed starting material and product profiles)
    sheet2_data = pd.DataFrame({
        'Time after addition (minutes)': time_after_addition_minutes,
        'Scaled Starting Material IR': scaled_starting_material_ir,
        'Scaled Product IR': scaled_product_ir
    })

    # Concentration profile for the second starting material
    if initial_conc_second_material!=None:
        second_material_concentration = initial_conc_second_material - scaled_product_ir
        if ArCl:
            second_material_concentration_free = initial_conc_second_material - scaled_product_ir*2
        #print('Second Material Concentration:', second_material_concentration)

    negative_time_indices = time_after_addition_minutes[time_after_addition_minutes < 0].index

    # Create DataFrame for Sheet 3 (concentration profiles)
    times_final_sheet=np.delete(time_after_addition_minutes, negative_time_indices[:-1])
    times_final_sheet[0]=0
    SM1_final_sheet=np.delete(scaled_starting_material_ir, negative_time_indices[:-1])
    SM1_final_sheet[0]=initial_conc_starting_material
    P_final_sheet=np.delete(scaled_product_ir, negative_time_indices[:-1])
    P_final_sheet[0]=0
    if initial_conc_second_material!=None:
        SM2_final_sheet=np.delete(second_material_concentration, negative_time_indices[:-1])
        SM2_final_sheet[0]=initial_conc_second_material
        if ArCl:
            SM2_final_sheet_free=np.delete(second_material_concentration_free, negative_time_indices[:-1])
            SM2_final_sheet_free[0]=initial_conc_second_material
    MB=SM1_final_sheet+P_final_sheet
    if initial_conc_second_material!=None:
        if ArCl:
            sheet3_data = pd.DataFrame({
                'Time / min': times_final_sheet,
                'Starting Material':SM1_final_sheet ,
                'Product': P_final_sheet,
                'Second Starting Material': SM2_final_sheet,
                'Second Starting Material free': SM2_final_sheet_free,
                'Mass Balance': MB
            })
        else:
            sheet3_data = pd.DataFrame({
                'Time / min': times_final_sheet,
                'Starting Material':SM1_final_sheet ,
                'Product': P_final_sheet,
                'Second Starting Material': SM2_final_sheet,
                'Mass Balance': MB
            })
    else:
            sheet3_data = pd.DataFrame({
                'Time / min': times_final_sheet,
                'Starting Material':SM1_final_sheet ,
                'Product': P_final_sheet,
                'Mass Balance': MB
            })
    if Name_SM1 not in ['',None]:
        sheet3_data.rename(columns={'Starting Material': Name_SM1},inplace=True)
    if Name_SM2 not in ['',None] and initial_conc_second_material!=None:
        sheet3_data.rename(columns={'Second Starting Material': Name_SM2},inplace=True)
        if ArCl:
            sheet3_data.rename(columns={'Second Starting Material free': Name_SM2+'free'},inplace=True)
            
    return sheet1_data, sheet2_data, sheet3_data

In [11]:
from datetime import datetime, timedelta, time
import pandas as pd
import numpy as np

def convert_string_time_to_seconds(time_value):
    """
    Convert a time string (e.g., 'HH:MM:SS' or 'MM:SS') or a datetime.time object to seconds.
    Returns NaN if the format is unrecognized.
    """
    if isinstance(time_value, str):
        try:
            parts = time_value.split(":")
            if len(parts) == 3:
                try:
                    time_obj = datetime.strptime(time_value, '%H:%M:%S.%f')
                except ValueError:
                    time_obj = datetime.strptime(time_value, '%H:%M:%S')
                return timedelta(hours=time_obj.hour, minutes=time_obj.minute, seconds=time_obj.second, microseconds=time_obj.microsecond).total_seconds()
            elif len(parts) == 2:
                try:
                    time_obj = datetime.strptime(time_value, '%M:%S.%f')
                except ValueError:
                    time_obj = datetime.strptime(time_value, '%M:%S')
                return timedelta(minutes=time_obj.minute, seconds=time_obj.second, microseconds=time_obj.microsecond).total_seconds()
        except ValueError:
            return np.nan
    elif isinstance(time_value, time):
        return timedelta(hours=time_value.hour, minutes=time_value.minute, seconds=time_value.second, microseconds=time_value.microsecond).total_seconds()
    return np.nan


def datetime_to_seconds(value):
    """
    Convert a datetime, time, timedelta, or string representation into total seconds.
    Raises ValueError on unsupported types.
    """
    if isinstance(value, datetime):
        base_date = datetime(1900, 1, 1)
        return (value - base_date).total_seconds()
    elif isinstance(value, str):
        try:
            dt_value = datetime.strptime(value, "%H:%M:%S.%f")
        except ValueError:
            dt_value = datetime.strptime(value, "%Y-%m-%d %H:%M:%S.%f")
        return (dt_value - datetime(1900, 1, 1)).total_seconds()
    elif isinstance(value, time):
        return timedelta(hours=value.hour, minutes=value.minute, seconds=value.second, microseconds=value.microsecond).total_seconds()
    elif isinstance(value, timedelta):
        return value.total_seconds()
    else:
        raise ValueError(f"Unsupported type: {type(value)}")


def convert_time_to_seconds(time_series):
    """
    Converts a pandas Series of datetime/time values to seconds,
    adjusting for day changes in multi-day experiments.
    """
    times_in_seconds = time_series.apply(datetime_to_seconds)
    corrected_times = []
    day_counter = 0
    for i, current_time in enumerate(times_in_seconds):
        if i > 0 and current_time < times_in_seconds[i - 1]:
            day_counter += 1
        corrected_times.append(current_time + 24 * 3600 * day_counter)
    return pd.Series(corrected_times)


def detect_starting_material_and_product(ir1, ir2):
    """
    Automatically determine which signal corresponds to starting material based on trends.
    Assumes starting material decreases and product increases over time.
    """
    if ir1.iloc[0] > ir1.iloc[-1]:
        return ir1, ir2
    return ir2, ir1


def process_ir_data(time_in_seconds, ir_sm, ir_p, reaction_start_time, ir_start_time,
                    conc_sm1, conc_sm2, name_sm1, name_sm2, time_original, ArCl=False):
    """
    Process IR data by aligning with reaction start, scaling based on known concentrations,
    and returning raw and processed datasets for plotting or export.
    """
    # Convert reference time points
    ir_start_sec = convert_string_time_to_seconds(ir_start_time)
    reaction_start_sec = convert_string_time_to_seconds(reaction_start_time)
    time_diff = reaction_start_sec - ir_start_sec
    print("Time difference between IR start and reaction start (in seconds):", time_diff)

    # Adjust times and convert to minutes
    adjusted_time = time_in_seconds + time_diff
    time_minutes = adjusted_time / 60
    #print("Adjusted time in minutes:", time_minutes)

    # Convert IR columns to numeric
    ir_sm = pd.to_numeric(ir_sm, errors='coerce')
    ir_p = pd.to_numeric(ir_p, errors='coerce')

    # Detect correct SM and product traces
    sm_ir, p_ir = detect_starting_material_and_product(ir_sm, ir_p)

    # Baseline values
    avg_final_sm_ir = sm_ir[-10:].mean()
    valid_sm_before = sm_ir[time_minutes < 0].tail(3).dropna()
    if len(valid_sm_before) < 3:
        raise ValueError("Not enough valid starting material IR values before reaction.")
    avg_init_sm_ir = valid_sm_before.mean()

    if pd.isna(avg_init_sm_ir) or avg_init_sm_ir == 0:
        raise ValueError("Invalid starting material IR average.")

    # Scale starting material IR
    sm_scaled = (sm_ir - avg_final_sm_ir) * (float(conc_sm1) / (avg_init_sm_ir - avg_final_sm_ir))

    # Scale product IR
    valid_p_before = p_ir[time_minutes < 0].tail(3).dropna()
    avg_init_p_ir = valid_p_before.mean()
    avg_final_p_ir = p_ir[-10:].mean()
    if pd.isna(avg_final_p_ir) or avg_final_p_ir == 0:
        raise ValueError("Invalid final product IR average.")
    p_scaled = (p_ir - avg_init_p_ir) * (float(conc_sm1) / (avg_final_p_ir - avg_init_p_ir))

    # Sheet 1: Raw data
    sheet1 = pd.DataFrame({
        'Time (HH:MM:SS)': time_original,
        'Time (minutes)': time_in_seconds / 60,
        'Time after addition (minutes)': time_minutes,
        'Starting Material IR': sm_ir,
        'Product IR': p_ir
    })

    # Sheet 2: Scaled profiles
    sheet2 = pd.DataFrame({
        'Time after addition (minutes)': time_minutes,
        'Scaled Starting Material IR': sm_scaled,
        'Scaled Product IR': p_scaled
    })

    # Compute second material concentration if given
    if conc_sm2 is not None:
        sm2_conc = float(conc_sm2) - p_scaled
        if ArCl:
            sm2_free = float(conc_sm2) - 2 * p_scaled

    # Prepare Sheet 3: Filter out all but last negative time point (set as zero)
    neg_indices = time_minutes[time_minutes < 0].index
    times_final = np.delete(time_minutes, neg_indices[:-1])
    times_final[0] = 0
    sm_final = np.delete(sm_scaled, neg_indices[:-1])
    sm_final[0] = float(conc_sm1)
    p_final = np.delete(p_scaled, neg_indices[:-1])
    p_final[0] = 0
    mb = sm_final + p_final

    if conc_sm2 is not None:
        sm2_final = np.delete(sm2_conc, neg_indices[:-1])
        sm2_final[0] = float(conc_sm2)
        if ArCl:
            sm2_free_final = np.delete(sm2_free, neg_indices[:-1])
            sm2_free_final[0] = float(conc_sm2)

    # Build sheet 3 DataFrame
    columns = {
        'Time / min': times_final,
        'Starting Material': sm_final,
        'Product': p_final,
        'Mass Balance': mb
    }

    if conc_sm2 is not None:
        columns['Second Starting Material'] = sm2_final
        if ArCl:
            columns['Second Starting Material free'] = sm2_free_final

    sheet3 = pd.DataFrame(columns)

    # Optional renaming
    if name_sm1:
        sheet3.rename(columns={'Starting Material': name_sm1}, inplace=True)
    if name_sm2 and conc_sm2 is not None:
        sheet3.rename(columns={'Second Starting Material': name_sm2}, inplace=True)
        if ArCl:
            sheet3.rename(columns={'Second Starting Material free': name_sm2 + ' free'}, inplace=True)

    return sheet1, sheet2, sheet3 


### Main Function for IR Scaling

In [12]:
# Main function to handle file reading, processing, and writing the output
def main(input_file, reaction_start_time, ir_start_time, initial_conc_starting_material, initial_conc_second_material, start_material_name='Starting Material', product_name='Product', second_material_name='Second Material'):
    file_name_without_ext = os.path.splitext(os.path.basename(input_file))[0]
    # Read the Excel file
    df = pd.read_excel(input_file)
    # Assuming the columns are: Time, Starting Material IR, Product IR
    time = df.iloc[:, 0]  # Convert time column to datetime 
    time_in_seconds = convert_time_to_seconds(time)
    ir_starting_material = df.iloc[:, 1]
    ir_product = df.iloc[:, 2]

    if start_material_name in ['ArCl','ArBr']:
        ArCl=True
    else:
        ArCl=False
    # Process the IR data
    sheet1_data, sheet2_data, sheet3_data = process_ir_data(
        time_in_seconds, ir_starting_material, ir_product, reaction_start_time, ir_start_time,
        initial_conc_starting_material, initial_conc_second_material,Name_SM1,Name_SM2,time,ArCl
    )

    # Write to an Excel file with three sheets
    output_file = f'{file_name_without_ext}_processed.xlsx'
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        sheet1_data.to_excel(writer, sheet_name='Raw IR Data', index=False)
        sheet2_data.to_excel(writer, sheet_name='Processed IR Profiles', index=False)
        sheet3_data.to_excel(writer, sheet_name='Concentration Profiles', index=False)
    
    print(f"Data successfully processed and saved to {output_file}")
    sheet3_csv_name = file_name_without_ext + '_data.csv'
    sheet3_data.to_csv(sheet3_csv_name, index=False)

### Explanation of Code

The code does the following:
- Time alignment: Aligns the IR time series with the actual reaction start time by calculating a time offset between the IR start and the reaction start. Time is converted into minutes.
- IR Trace Assignment: Automatically determines which of the two IR traces corresponds to the starting material (SM) and which to the product (P), based on signal trends (e.g., SM decreases, P increases).
- Signal Scaling: Scales both IR traces into concentrations:
    - The starting material IR signal is scaled based on initial and final baselines and a known starting concentration.
    - The product IR signal is scaled similarly, using its baseline values before and after the reaction.
- Optional Second Material Calculation:If a second reactant concentration is provided, the function calculates its decreasing concentration as the reaction proceeds. For aryl chlorides and bromides, the concentration of the free non-protonated amine is also calculated by subtracting two equivalents of product from the initial concentration at each time point. However, this is not necessary for aryl phenyl ether electrophiles.

The script then outputs two files:
- An xlsx. file with three sheets showing the progression of the data treatment:
    - 1: Raw IR trends with a time axis (minutes) set to 0 at the start of the reaction (first amine addition) rather than the start of the IR experiment.
    - 2: Scaled IR data (from inputted initial concentration of electrophile, assuming complete conversion to product. Note that for SAKE experiments, this is somewhat incorrect as the final product concentration is lower than that of the initial electrophile concentration due to dilution via additions made over the experiment.
    - 3: Cleaned and filtered data, with time starting at 0 (last IR recorded before start of experiment) and with a mass balance column showing the sum of the product and electrophile concentrations across time.
- A csv file containing the third sheet of the xlsx. file. This is used as an input for the proper IR scaling done in the Jupyter Notebook script Scaling of IR trends with Calibrated HPLC data.ipynb in the folder Scaling of IR trends with calibrated HPLC Data.

### Example of Use for SAKE Experiments (First 2 Experiments of Campaign 1)

In [13]:
path_to_trend_inputs=os.getcwd()+"\\IR trend input data"

In [14]:
Experiments=[]
for file in os.listdir(path_to_trend_inputs):
    Experiments.append(file[:-14].replace(' ',''))

In [15]:
# Example usage
if __name__ == "__main__":
    input_file = path_to_trend_inputs+'\\C1_E1 IR trends.xlsx' 
    reaction_start_time = '09:33:54'  
    ir_start_time = '09:25:15' 
    initial_conc_starting_material =121.427473
    initial_conc_second_material = None  
    Name_SM1='4SO2Me-ArOPh'
    Name_SM2='4MePip'

    main(input_file, ir_start_time,reaction_start_time, initial_conc_starting_material, initial_conc_second_material,Name_SM1,Name_SM2)


Time difference between IR start and reaction start (in seconds): -519.0
Data successfully processed and saved to C1_E1 IR trends_processed.xlsx


In [16]:
# Example usage
if __name__ == "__main__":
    input_file = path_to_trend_inputs+'\\C1_E2 IR trends.xlsx'  
    reaction_start_time = '15:48:02'  
    ir_start_time = '15:28:50'  
    initial_conc_starting_material =124.5646823
    initial_conc_second_material = None  
    Name_SM1='4SO2Me-ArOPh'
    Name_SM2='4MePip'

    main(input_file, ir_start_time,reaction_start_time, initial_conc_starting_material, initial_conc_second_material,Name_SM1,Name_SM2)


Time difference between IR start and reaction start (in seconds): -1152.0
Data successfully processed and saved to C1_E2 IR trends_processed.xlsx
